# Tutorial 07: understanding restrictions

In [ ]:
import matplotlib
import matplotlib.collections
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from dolfinx import FunctionSpace, MPI
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import DofMapRestriction, locate_dofs_geometrical, locate_dofs_topological
from dolfinx.io import XDMFFile

### Auxiliary functions for plotting

In [ ]:
def plot_mesh(mesh, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    points = mesh.geometry.x
    tria = tri.Triangulation(points[:, 0], points[:, 1], mesh.cells())
    ax.triplot(tria, color="k")
    return ax

In [ ]:
def plot_mesh_function(mesh_function, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    mesh = mesh_function.mesh()
    points = mesh.geometry.x
    colors = ["b", "r"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap_bounds = [0, 0.5, 1]
    norm = matplotlib.colors.BoundaryNorm(cmap_bounds, cmap.N)
    assert mesh_function.dim in (mesh.topology.dim, mesh.topology.dim - 1)
    if mesh_function.dim == mesh.topology.dim:
        cells = mesh.cells()
        tria = tri.Triangulation(points[:, 0], points[:, 1], cells)
        cell_colors = np.zeros((cells.shape[0], ))
        for v in (0, 1):
            cell_colors[np.where(mesh_function.values == v)[0]] = v
        mappable = ax.tripcolor(tria, cell_colors, edgecolor="k", cmap=cmap, norm=norm)
    elif mesh_function.dim == mesh.topology.dim - 1:
        tdim = mesh.topology.dim
        connectivity_cells_to_facets = mesh.topology.connectivity(tdim, tdim - 1)
        connectivity_facets_to_vertices = mesh.topology.connectivity(tdim - 1, 0)
        linestyles = [(0, (5, 10)), "solid"]
        lines = list()
        lines_colors_as_int = list()
        lines_colors_as_str = list()
        lines_linestyles = list()
        for c in range(mesh.num_cells()):
            facets = connectivity_cells_to_facets.links(c)
            for f in facets:
                mesh_function_f = mesh_function.values[f]
                vertices = connectivity_facets_to_vertices.links(f)
                lines.append(points[vertices][:, :2])
                lines_colors_as_int.append(mesh_function_f)
                lines_colors_as_str.append(colors[mesh_function_f])
                lines_linestyles.append(linestyles[mesh_function_f])
        mappable = matplotlib.collections.LineCollection(lines, cmap=cmap, norm=norm,
                                                         colors=lines_colors_as_str,
                                                         linestyles=lines_linestyles)
        mappable.set_array(np.array(lines_colors_as_int))
        ax.add_collection(mappable)
        ax.autoscale()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(mappable, cax=cax, cmap=cmap, norm=norm, boundaries=cmap_bounds, ticks=cmap_bounds)
    return ax

In [ ]:
def _plot_dofmap(coordinates, ax=None):
    if ax is None:
        ax = plt.gca()
    text_offset = [1e-2, 1e-2]
    ax.scatter(coordinates[:, 0], coordinates[:, 1], c="k", s=50)
    for c in np.unique(coordinates, axis=0):
        dofs_c = (coordinates == c).all(axis=1).nonzero()[0]
        text_c = np.array2string(dofs_c, separator=", ", max_line_width=10)
        ax.text(c[0] + text_offset[0], c[1] + text_offset[1], text_c, fontsize=20)
    return ax

In [ ]:
def plot_dofmap(V, ax=None):
    coordinates = V.tabulate_dof_coordinates().round(decimals=3)
    return _plot_dofmap(coordinates, ax)

In [ ]:
def plot_dofmap_restriction(V, restriction, ax=None):
    coordinates = V.tabulate_dof_coordinates().round(decimals=3)
    return _plot_dofmap(coordinates[list(restriction.unrestricted_to_restricted.keys())], ax)

### Auxiliary functions for asserts

In [ ]:
def count_dofs(restriction):
    u2r = restriction.unrestricted_to_restricted
    dofs_V_restriction_global = [restriction.index_map.local_to_global(dof) for (_, dof) in u2r.items()]
    return len(set(gdof
                   for gdofs in restriction.index_map.mpi_comm().allgather(dofs_V_restriction_global)
                   for gdof in gdofs))

In [ ]:
def locate_dofs_by_polar_coordinates(r, theta, V, restriction):
    p = [r * np.cos(theta), r * np.sin(theta), 0.]
    dofs_V = locate_dofs_geometrical(V, lambda x: np.isclose(x.T, p).all(axis=1)).reshape(-1)
    u2r = restriction.unrestricted_to_restricted
    dofs_V_restriction = [u2r[dof] for dof in dofs_V if dof in u2r]
    dofs_V_restriction_global = [restriction.index_map.local_to_global(dof)
                                 for dof in dofs_V_restriction]
    return set(gdof
               for gdofs in restriction.index_map.mpi_comm().allgather(dofs_V_restriction_global)
               for gdof in gdofs)

### Mesh

Read in a simple mesh consisting in an hexagon discretized with six equilateral triangle cells.

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)

In [ ]:
plot_mesh(mesh)

### Mesh restrictions on cells

Read in a mesh function on cells, which is equal to one on all cells.

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_cell_restriction_all.xdmf") as infile:
    cell_restriction_all = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(cell_restriction_all)

Read in a mesh function on cells, which is equal to zero on one half of the cells, and equal to one on the other half of the cells

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_cell_restriction_subset.xdmf") as infile:
    cell_restriction_subset = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(cell_restriction_subset)

### Mesh restrictions on facets

Define a mesh function on facets, which is equal to one on all facets

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_facet_restriction_all.xdmf") as infile:
    facet_restriction_all = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(facet_restriction_all)

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_facet_restriction_subset.xdmf") as infile:
    facet_restriction_subset = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(facet_restriction_subset)

In [ ]:
cell_restrictions = (cell_restriction_all, cell_restriction_subset)
facet_restrictions = (facet_restriction_all, facet_restriction_subset)
all_restrictions = cell_restrictions + facet_restrictions

### Lagrange spaces

Define Lagrange FE spaces of order $k=1, 2, 3$, and plot the associated DofMap.

In [ ]:
CG = [FunctionSpace(mesh, ("Lagrange", k)) for k in (1, 2, 3)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k])
    plot_dofmap(CGk, ax[k])
    ax[k].set_title("CG " + str(k + 1) + " DofMap", fontsize=30)

Define DofMapRestriction objects associated to the Lagrange FE spaces, for all four restrictions

In [ ]:
dofmap_restriction_CG = dict()
for restriction in all_restrictions:
    dofmap_restriction_CG[restriction] = list()
    for CGk in CG:
        restrict_CGk = locate_dofs_topological(CGk, restriction.dim, np.where(restriction.values == 1)[0])
        dofmap_restriction_CG[restriction].append(DofMapRestriction(CGk.dofmap, restrict_CGk))

Compare DOFs for the case of cell restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CGk, ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " DofMap", fontsize=30)
for (k, (CGk, dofmap_restriction_CGk)) in enumerate(zip(CG, dofmap_restriction_CG[cell_restriction_all])):
    plot_mesh_function(cell_restriction_all, ax[k, 1])
    plot_dofmap_restriction(CGk, dofmap_restriction_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
CG_1 = CG[0]
dofmap_restriction_CG_1 = dofmap_restriction_CG[cell_restriction_all][0]
assert count_dofs(dofmap_restriction_CG_1) == 7
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1

In [ ]:
CG_2 = CG[1]
dofmap_restriction_CG_2 = dofmap_restriction_CG[cell_restriction_all][1]
assert count_dofs(dofmap_restriction_CG_2) == 19
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 3 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 11 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1

In [ ]:
CG_3 = CG[2]
dofmap_restriction_CG_3 = dofmap_restriction_CG[cell_restriction_all][2]
assert count_dofs(dofmap_restriction_CG_3) == 37
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 3 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 5 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 7 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 9 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 11 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 2 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 2 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 5 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 5 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1

Compare DOFs for che case of cell restriction equal to one on a subset of the domain. Note how the DofMapRestriction has only a subset of the DOFs of the DofMap, and properly renumbers them.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CGk, ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " DofMap", fontsize=30)
for (k, (CGk, dofmap_restriction_CGk)) in enumerate(zip(CG, dofmap_restriction_CG[cell_restriction_subset])):
    plot_mesh_function(cell_restriction_subset, ax[k, 1])
    plot_dofmap_restriction(CGk, dofmap_restriction_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
CG_1 = CG[0]
dofmap_restriction_CG_1 = dofmap_restriction_CG[cell_restriction_subset][0]
assert count_dofs(dofmap_restriction_CG_1) == 5
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1

In [ ]:
CG_2 = CG[1]
dofmap_restriction_CG_2 = dofmap_restriction_CG[cell_restriction_subset][1]
assert count_dofs(dofmap_restriction_CG_2) == 12
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1

In [ ]:
CG_3 = CG[2]
dofmap_restriction_CG_3 = dofmap_restriction_CG[cell_restriction_subset][2]
assert count_dofs(dofmap_restriction_CG_3) == 22
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 5 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 7 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 9 * np.pi / 6, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 2 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 5 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1

Compare DOFs for che case of facet restriction equal to one on the entire domain. Note how there is no difference for $k=1, 2$, but the cases $k=3$ differ (the DofMapRestriction does not have the DOF at the cell center).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CG[k], ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " DofMap", fontsize=30)
for (k, (CGk, dofmap_restriction_CGk)) in enumerate(zip(CG, dofmap_restriction_CG[facet_restriction_all])):
    plot_mesh_function(facet_restriction_all, ax[k, 1])
    plot_dofmap_restriction(CGk, dofmap_restriction_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
CG_1 = CG[0]
dofmap_restriction_CG_1 = dofmap_restriction_CG[facet_restriction_all][0]
assert count_dofs(dofmap_restriction_CG_1) == 7
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1

In [ ]:
CG_2 = CG[1]
dofmap_restriction_CG_2 = dofmap_restriction_CG[facet_restriction_all][1]
assert count_dofs(dofmap_restriction_CG_2) == 19
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 3 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 11 * np.pi / 6, CG_2, dofmap_restriction_CG_2)) == 1

In [ ]:
CG_3 = CG[2]
dofmap_restriction_CG_3 = dofmap_restriction_CG[facet_restriction_all][2]
assert count_dofs(dofmap_restriction_CG_3) == 31
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 2 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 2 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 4 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 5 * np.pi / 3 - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, 5 * np.pi / 3 + np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(7) / 3, np.pi - np.arctan(np.sqrt(3) / 5), CG_3, dofmap_restriction_CG_3)) == 1

Compare DOFs for che case of facet restriction equal to one on a subset of the domain. Note how the DofMapRestriction has only a subset of the DOFs of the DofMap, and properly renumbers them.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CG[k], ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " DofMap", fontsize=30)
for (k, (CGk, dofmap_restriction_CGk)) in enumerate(zip(CG, dofmap_restriction_CG[facet_restriction_subset])):
    plot_mesh_function(facet_restriction_subset, ax[k, 1])
    plot_dofmap_restriction(CGk, dofmap_restriction_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
CG_1 = CG[0]
dofmap_restriction_CG_1 = dofmap_restriction_CG[facet_restriction_subset][0]
assert count_dofs(dofmap_restriction_CG_1) == 3
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_1, dofmap_restriction_CG_1)) == 1

In [ ]:
CG_2 = CG[1]
dofmap_restriction_CG_2 = dofmap_restriction_CG[facet_restriction_subset][1]
assert count_dofs(dofmap_restriction_CG_2) == 5
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_2, dofmap_restriction_CG_2)) == 1

In [ ]:
CG_3 = CG[2]
dofmap_restriction_CG_3 = dofmap_restriction_CG[facet_restriction_subset][2]
assert count_dofs(dofmap_restriction_CG_3) == 7
assert len(locate_dofs_by_polar_coordinates(0, 0, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(2 / 3, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, CG_3, dofmap_restriction_CG_3)) == 1

### Discontinuous Galerkin spaces

Define Discontinuous Galerkin spaces of order $k = 0, 1, 2$, and plot the associated DofMap. This spaces will be used in combination with a cell restriction, as DG DOFs are associated to cells.

In [ ]:
DG = [FunctionSpace(mesh, ("Discontinuous Lagrange", k)) for k in (0, 1, 2)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k])
    plot_dofmap(DGk, ax[k])
    ax[k].set_title("DG " + str(k) + " DofMap", fontsize=30)

Define Discontinuous Galerkin Trace spaces of order $k = 0, 1, 2, 3$, and plot the associated DofMap. This spaces will be used in combination with a facet restriction, as DGT DOFs are associated to facets.

In [ ]:
DGT = [FunctionSpace(mesh, ("Discontinuous Lagrange Trace", k)) for k in (0, 1, 2)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k])
    plot_dofmap(DGTk, ax[k])
    ax[k].set_title("DGT " + str(k) + " DofMap\n", fontsize=30)

Define DofMapRestriction objects associated to the Discontinuos Galerkin FE spaces, for all cell restrictions

In [ ]:
dofmap_restriction_DG = dict()
for restriction in cell_restrictions:
    dofmap_restriction_DG[restriction] = list()
    for DGk in DG:
        restrict_DGk = locate_dofs_topological(DGk, restriction.dim, np.where(restriction.values == 1)[0])
        dofmap_restriction_DG[restriction].append(DofMapRestriction(DGk.dofmap, restrict_DGk))

Define DofMapRestriction objects associated to the Discontinuos Galerkin Trace FE spaces, for all facet restrictions

In [ ]:
dofmap_restriction_DGT = dict()
for restriction in facet_restrictions:
    dofmap_restriction_DGT[restriction] = list()
    for DGTk in DGT:
        restrict_DGTk = locate_dofs_topological(DGTk, restriction.dim, np.where(restriction.values == 1)[0])
        dofmap_restriction_DGT[restriction].append(DofMapRestriction(DGTk.dofmap, restrict_DGTk))

Compare DOFs for the case of cell restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGk, ax[k, 0])
    ax[k, 0].set_title("DG " + str(k) + " DofMap", fontsize=30)
for (k, (DGk, dofmap_restriction_DGk)) in enumerate(zip(DG, dofmap_restriction_DG[cell_restriction_all])):
    plot_mesh_function(cell_restriction_all, ax[k, 1])
    plot_dofmap_restriction(DGk, dofmap_restriction_DGk, ax[k, 1])
    ax[k, 1].set_title("DG " + str(k) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
DG_0 = DG[0]
dofmap_restriction_DG_0 = dofmap_restriction_DG[cell_restriction_all][0]
assert count_dofs(dofmap_restriction_DG_0) == 6
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 3 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 5 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 7 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 9 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 11 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1

In [ ]:
DG_1 = DG[1]
dofmap_restriction_DG_1 = dofmap_restriction_DG[cell_restriction_all][1]
assert count_dofs(dofmap_restriction_DG_1) == 18
assert len(locate_dofs_by_polar_coordinates(0, 0, DG_1, dofmap_restriction_DG_1)) == 6
assert len(locate_dofs_by_polar_coordinates(1, 0, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 2

In [ ]:
DG_2 = DG[2]
dofmap_restriction_DG_2 = dofmap_restriction_DG[cell_restriction_all][2]
assert count_dofs(dofmap_restriction_DG_2) == 36
assert len(locate_dofs_by_polar_coordinates(0, 0, DG_2, dofmap_restriction_DG_2)) == 6
assert len(locate_dofs_by_polar_coordinates(0.5, 0, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 0, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 3 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 11 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1

Compare DOFs for che case of cell restriction equal to one on a subset of the domain. Note how the DofMapRestriction has only a subset of the DOFs of the DofMap, and properly renumbers them. Note also that the number of DOFs at the same physical location might be different between DofMap and DofMapRestriction (see e.g. the center of the hexagon).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGk, ax[k, 0])
    ax[k, 0].set_title("DG " + str(k) + " DofMap", fontsize=30)
for (k, (DGk, dofmap_restriction_DGk)) in enumerate(zip(DG, dofmap_restriction_DG[cell_restriction_subset])):
    plot_mesh_function(cell_restriction_subset, ax[k, 1])
    plot_dofmap_restriction(DGk, dofmap_restriction_DGk, ax[k, 1])
    ax[k, 1].set_title("DG " + str(k) + " DofMapRestriction", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
DG_0 = DG[0]
dofmap_restriction_DG_0 = dofmap_restriction_DG[cell_restriction_subset][0]
assert count_dofs(dofmap_restriction_DG_0) == 3
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 5 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 7 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 3, 9 * np.pi / 6, DG_0, dofmap_restriction_DG_0)) == 1

In [ ]:
DG_1 = DG[1]
dofmap_restriction_DG_1 = dofmap_restriction_DG[cell_restriction_subset][1]
assert count_dofs(dofmap_restriction_DG_1) == 9
assert len(locate_dofs_by_polar_coordinates(0, 0, DG_1, dofmap_restriction_DG_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DG_1, dofmap_restriction_DG_1)) == 1

In [ ]:
DG_2 = DG[2]
dofmap_restriction_DG_2 = dofmap_restriction_DG[cell_restriction_subset][2]
assert count_dofs(dofmap_restriction_DG_2) == 18
assert len(locate_dofs_by_polar_coordinates(0, 0, DG_2, dofmap_restriction_DG_2)) == 3
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, DG_2, dofmap_restriction_DG_2)) == 1

Compare DOFs for che case of facet restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGTk, ax[k, 0])
    ax[k, 0].set_title("DGT " + str(k) + " DofMap\n", fontsize=30)
for (k, (DGTk, dofmap_restriction_DGTk)) in enumerate(zip(DGT, dofmap_restriction_DGT[facet_restriction_all])):
    plot_mesh_function(facet_restriction_all, ax[k, 1])
    plot_dofmap_restriction(DGTk, dofmap_restriction_DGTk, ax[k, 1])
    ax[k, 1].set_title("DGT " + str(k) + " DofMapRestriction\n", fontsize=30)

Assert that DOFs are at the expected locations

In [ ]:
DGT_0 = DGT[0]
dofmap_restriction_DGT_0 = dofmap_restriction_DGT[facet_restriction_all][0]
assert count_dofs(dofmap_restriction_DGT_0) == 12
assert len(locate_dofs_by_polar_coordinates(0.5, 0, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 3 * np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 11 * np.pi / 6, DGT_0, dofmap_restriction_DGT_0)) == 1

In [ ]:
DGT_1 = DGT[1]
dofmap_restriction_DGT_1 = dofmap_restriction_DGT[facet_restriction_all][1]
assert count_dofs(dofmap_restriction_DGT_1) == 24
assert len(locate_dofs_by_polar_coordinates(0, 0, DGT_1, dofmap_restriction_DGT_1)) == 6
assert len(locate_dofs_by_polar_coordinates(1, 0, DGT_1, dofmap_restriction_DGT_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DGT_1, dofmap_restriction_DGT_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 3

In [ ]:
DGT_2 = DGT[2]
dofmap_restriction_DGT_2 = dofmap_restriction_DGT[facet_restriction_all][2]
assert count_dofs(dofmap_restriction_DGT_2) == 36
assert len(locate_dofs_by_polar_coordinates(0, 0, DGT_2, dofmap_restriction_DGT_2)) == 6
assert len(locate_dofs_by_polar_coordinates(0.5, 0, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, np.pi, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 4 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 0, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(1, np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(1, np.pi, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 4 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 3
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 3 * np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 5 * np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 7 * np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 9 * np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(np.sqrt(3) / 2, 11 * np.pi / 6, DGT_2, dofmap_restriction_DGT_2)) == 1

Compare DOFs for che case of facet restriction equal to one on a subset of the domain. Note how the DofMapRestriction has only a subset of the DOFs of the DofMap, and properly renumbers them. Note also that the number of DOFs at the same physical location might be different between DofMap and DofMapRestriction (see e.g. the center of the hexagon).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGTk, ax[k, 0])
    ax[k, 0].set_title("DGT " + str(k) + " DofMap\n", fontsize=30)
for (k, (DGTk, dofmap_restriction_DGTk)) in enumerate(zip(DGT, dofmap_restriction_DGT[facet_restriction_subset])):
    plot_mesh_function(facet_restriction_subset, ax[k, 1])
    plot_dofmap_restriction(DGTk, dofmap_restriction_DGTk, ax[k, 1])
    ax[k, 1].set_title("DGT " + str(k) + " DofMapRestriction\n", fontsize=30)

Assert that DOFfs are at the expected locations

In [ ]:
DGT_0 = DGT[0]
dofmap_restriction_DGT_0 = dofmap_restriction_DGT[facet_restriction_subset][0]
assert count_dofs(dofmap_restriction_DGT_0) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DGT_0, dofmap_restriction_DGT_0)) == 1

In [ ]:
DGT_1 = DGT[1]
dofmap_restriction_DGT_1 = dofmap_restriction_DGT[facet_restriction_subset][1]
assert count_dofs(dofmap_restriction_DGT_1) == 4
assert len(locate_dofs_by_polar_coordinates(0, 0, DGT_1, dofmap_restriction_DGT_1)) == 2
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DGT_1, dofmap_restriction_DGT_1)) == 1

In [ ]:
DGT_2 = DGT[2]
dofmap_restriction_DGT_2 = dofmap_restriction_DGT[facet_restriction_subset][2]
assert count_dofs(dofmap_restriction_DGT_2) == 6
assert len(locate_dofs_by_polar_coordinates(0, 0, DGT_2, dofmap_restriction_DGT_2)) == 2
assert len(locate_dofs_by_polar_coordinates(0.5, 2 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(0.5, 5 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 2 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1
assert len(locate_dofs_by_polar_coordinates(1, 5 * np.pi / 3, DGT_2, dofmap_restriction_DGT_2)) == 1